In [6]:
import quantiacsToolbox
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [9]:
class myStrategy(object):

    
    def myTradingSystem(self,DATE, OPEN, HIGH, LOW, CLOSE, VOL, OI, P, R, RINFO, exposure, equity, settings):
        
        def RSI(CLOSE, period):
            closeMom = CLOSE[1:,:] - CLOSE[:-1,:]
            up = closeMom >= 0
            down = closeMom < 0
            out = 100 - 100/(1 + (np.mean(up[-(period+1):,:], axis=0)/ np.mean(down[-(period+1):,:], axis=0)))
            return out
       
        nMarkets = len(settings['markets'])
        pos = np.zeros((1,nMarkets))
        pos_1 = np.zeros((1,nMarkets))
        dur = settings['duration']
        rsi1 = RSI(CLOSE,100)
        rsi2 = RSI(CLOSE,50)
        vol_trend =  np.mean(VOL[-5:,:], axis=0)/np.mean(VOL[-20:,:], axis=0)   #np.mean((HIGH[-20:,:]+LOW[-20:,:]+OPEN[-20:,:]+CLOSE[-20,:])/4, axis=0)
        price_trend = np.mean(CLOSE[-5:,:], axis=0)/np.mean(CLOSE[-20:,:], axis=0)
        #pos = (avg > last*1.08)?1:(avg < last*0.95)?-1:0
        for i in range(nMarkets):
            pos_1[0,i] = (np.log(vol_trend[i])+np.log(price_trend[i]))*np.sign(-np.mean(CLOSE[-5:,i], axis=0) + np.mean(CLOSE[-20:,i], axis=0))
            
        pos_rsi = ((rsi1-50) + (rsi2-50))/2
    
        pos_1 = 0.5*pos_rsi + pos_1*0.5  
        pos_1[np.isinf(pos_1)]=0
#         pos[pos<0] = pos[pos<0]/10
        for i in range(nMarkets):
            close = (CLOSE[-dur:, i]).reshape(-1,1)
            #close = close[~np.isnan(close)]
            vol = (VOL[-dur:, i]).reshape(-1,1)
            #vol = vol[~np.isnan(vol)]
            x = np.append(close, vol, axis=1)
            #print(x)
            x = x[~np.isnan(x).any(axis=1),:]
            #print(x.shape)
            [n,m] = x.shape
            if n>0:
                vwap = np.sum(np.multiply(x[:,0],x[:,1]))/np.sum(x[:,1])
                alpha = (vwap/x[-1,0])
                pos[0,i] = alpha
            else:
                pos[0,i] = 0
            #p = 100*np.log(pos)
            #sign_pos = pos>0
            pos[np.isnan(pos)] = 0
            pos = pos + pos_1
            #num_elm = np.count_nonzero(pos)
            #print(num_elm)
            #print(pos)
        return pos, settings

    
    def mySettings(self):
        """ Define your trading system settings here """

        settings = {}

        # Futures Contracts
        settings['markets'] = ['CASH', 'F_AD', 'F_AE', 'F_AH', 'F_AX', 'F_BC', 'F_BG', 'F_BO', 'F_BP', 'F_C',  'F_CA',
                               'F_CC', 'F_CD', 'F_CF', 'F_CL', 'F_CT', 'F_DL', 'F_DM', 'F_DT', 'F_DX', 'F_DZ', 'F_EB',
                               'F_EC', 'F_ED', 'F_ES', 'F_F',  'F_FB', 'F_FC', 'F_FL', 'F_FM', 'F_FP', 'F_FV', 'F_FY',
                               'F_GC', 'F_GD', 'F_GS', 'F_GX', 'F_HG', 'F_HO', 'F_HP', 'F_JY', 'F_KC', 'F_LB', 'F_LC',
                               'F_LN', 'F_LQ', 'F_LR', 'F_LU', 'F_LX', 'F_MD', 'F_MP', 'F_ND', 'F_NG', 'F_NQ', 'F_NR',
                               'F_NY', 'F_O',  'F_OJ', 'F_PA', 'F_PL', 'F_PQ', 'F_RB', 'F_RF', 'F_RP', 'F_RR', 'F_RU',
                               'F_RY', 'F_S',  'F_SB', 'F_SF', 'F_SH', 'F_SI', 'F_SM', 'F_SS', 'F_SX', 'F_TR', 'F_TU',
                               'F_TY', 'F_UB', 'F_US', 'F_UZ', 'F_VF', 'F_VT', 'F_VW', 'F_VX',  'F_W', 'F_XX', 'F_YM',
                              'F_ZQ']
        #markets to  decide:
        #currency -> 'US-Dollar Index', 'euro_fx', 'jap_yen', 'safrica rand', 
        #agri -> 'corn', 'cotton', coffee, sugar, wheat, rice, 
        #index -> volatility index, bloomberg commodity index, russel200, dax, cac 
        #energy -> nat gas,brent crude, newcastle coal,
        #int rate -> 30 day fed, eurodollar
        #bond-> 2 yr tre, 10 yr, 30 yr, euro bond
        #metal -> gold, cu, 
        #stocks -> NA
#         settings['markets'] = ['F_DX', 'F_EC', 'F_JY', 'F_TR', 
#                                 'F_C', 'F_CT', 'F_KC', 'F_SB', 'F_W',  'F_NR', 
#                                 'F_VX', 'F_AH', 'F_RU', 'F_AX', 'F_CA', 
#                                 'F_NG', 'F_BC', 'F_LQ',
#                                 'F_ZQ', 'F_ED', 
#                                 'F_TU', 'F_TY', 'F_US', 'F_DT',  
#                                 'F_GC', 'F_HG']
        settings['lookback'] = 504
        settings['budget'] = 10 ** 6
        settings['slippage'] = 0.05

        settings['duration'] = 20

        return settings

In [10]:
result = quantiacsToolbox.runts(myStrategy)

Loading Data...
Done!
Evaluating Trading System <class '__main__.myStrategy'>


/home/anshul/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater_equal
  
/home/anshul/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in less
  if __name__ == '__main__':
/home/anshul/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/anshul/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/anshul/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
/home/anshul/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/home/anshul/anaconda3/lib/python3.6/site-packages/ipykerne